In [ ]:
# Start time of script execution
ExecuteStart = system.date.now()
print "Execute Script Start Time: ", ExecuteStart

# Define start and end dates
sDate = system.date.getDate(2024, 1, 01)
eDate = system.date.getDate(2024, 1, 10)

# Format dates to a readable format
startDate = system.date.format(sDate, "yyyy-MM-dd HH:mm:ss")
endDate = system.date.format(eDate, "yyyy-MM-dd HH:mm:ss")  

# Calculate the number of days between the dates
numDays = system.date.daysBetween(sDate, eDate)
timeout = numDays * 1000 * 24 * 12  # Set timeout based on days

# Set return size for 1-minute intervals
returnSize = numDays * 24 * 60

# Print debug info
print "Start Date: ", startDate
print "End Date: ", endDate
print "Number of Days: ", numDays
print "Script Timeout: ", timeout
print "Return Size: ", returnSize

# Define paths to query
paths = [
    "Y",
]

# Define aggregation modes
aggregationModes = [
    "Z",  # Change to other modes like "MinMax" based on need
]

# Query historical data
data = system.tag.queryTagHistory(
    paths=paths, 
    startDate=startDate, 
    endDate=endDate, 
    returnSize=returnSize, 
    aggregationModes=aggregationModes, 
    returnFormat='Tall', 
    timeout=timeout
)

# Extract column names as a proper list of strings
columnNames = list(data.getColumnNames())  # Ensure this is a list of strings
print "Column Names: ", columnNames

# Convert the dataset to a Python dataset for manipulation
pyDataSet = system.dataset.toPyDataSet(data)
""" 
# Assuming `pyDataSet` is your Python dataset
for i in range(min(5, len(pyDataSet))):  # Limit to 5 rows or the total number of rows if less than 5
    column_types = [type(value) for value in pyDataSet[i]]  # Get data types of each column in the row
    print("Row " + str(i + 1) + " types: " + str(column_types))  # Print the list of data types for the current row
"""

# Round numeric values to 2 decimal places
roundedRows = [
    [round(value, 2) if isinstance(value, (float, int)) else value for value in row]
    for row in pyDataSet
]



# Loop through the dataset and insert each row into the database

for row in roundedRows:
    tag_name = row[0]  # First column: tag path (or tag name)
    value = row[1]     # Second column: value
    T_stamp = row[3]  # 4th column: timestamp
    
    if value is not None:  # Avoid inserting NULL values
        # Insert the data into the PostgreSQL database with UPSERT to avoid duplicates
        query = """
            INSERT INTO records (T_stamp, tag_name, value)
            VALUES (?, ?, ?)
            ON CONFLICT (T_stamp, tag_name)
            DO UPDATE SET value = EXCLUDED.value;
        """
        system.db.runPrepUpdate(query, [T_stamp, tag_name, value], "W")

# End time of script execution
ExecuteEnd = system.date.now()
print "Execute Script End Time: ", ExecuteEnd
print "It took ", system.date.secondsBetween(ExecuteStart, ExecuteEnd), " seconds to execute"
